In [1]:
import requests
import pandas as pd
import datetime as dt
import time

In [2]:
endpoint = 'wss://stream.binance.com:9443/ws'
symbol = 'ethusdt'
symbol_C = symbol.upper()
interval = '15m'

# 2021/1/1
start_time = 1609459200000
end_time = round(time.time() * 1000)

# step between timestamps in milliseconds
step = 60000 * 3600

In [3]:
def get_historical(symbol, interval, start_time, end_time, step):
    
    raw_df = pd.DataFrame()
    
    url = "https://api.binance.com/api/v3/klines"
    
    for timestamp in range(start_time, end_time, step):
        params = {"symbol": symbol_C,
                  "interval": interval,
                  "startTime": timestamp,
                  "endTime": timestamp + step}
        response = requests.get(url, params=params).json()
        out = pd.DataFrame(response, columns = ["Open time", "Open", "High", "Low", "Close",
                                               "Volume", "Close_Time", "Quote asset volume",
                                               "Number of trades", "Taker buy base asset volume",
                                               "Taker buy quote asset volume", "Ignore"])
        raw_df = pd.concat([raw_df, out], axis = 0)
    
    raw_df = raw_df[['Close_Time', 'Open', 'Close', "High", "Low", 'Volume']]

    raw_df.to_hdf(f'klines_{symbol}_{interval}.h5', key='df', mode='w')

In [4]:
get_historical(symbol, interval, start_time, end_time, step)

In [5]:
df = pd.read_hdf(f'klines_{symbol}_{interval}.h5', key='df')
print(df)

        Close_Time           Open          Close           High  \
0    1609460099999   736.42000000   731.14000000   739.00000000   
1    1609460999999   731.14000000   731.60000000   732.28000000   
2    1609461899999   731.55000000   733.54000000   734.29000000   
3    1609462799999   733.54000000   734.07000000   735.59000000   
4    1609463699999   734.08000000   743.41000000   743.47000000   
..             ...            ...            ...            ...   
163  1680238799999  1804.33000000  1804.71000000  1805.46000000   
164  1680239699999  1804.71000000  1802.23000000  1808.50000000   
165  1680240599999  1802.23000000  1802.89000000  1803.84000000   
166  1680241499999  1802.90000000  1802.31000000  1803.92000000   
167  1680242399999  1802.30000000  1796.18000000  1802.61000000   

               Low          Volume  
0     730.03000000  10679.63623000  
1     729.33000000   6954.72694000  
2     731.45000000   4958.21322000  
3     732.38000000   5340.12245000  
4     733.